# **Using CapsNet for Video Classification**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

## **Load the datasets**

In [ ]:
# Initialize labels
LABELS = set(["Abuse", "Assault", "Fighting", "Normal", "Robbery", "Vandalism"])

# Initialize the list of images
print("Loading images:")
imagePaths = list(paths.list_images(r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Datasets'))

data = []
labels = []

# Loop over the image paths
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]

    if label not in LABELS:
        continue
    
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    data.append(image)
    labels.append(label)

np.array(labels)
np.array(data)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

# Initialize the training data augmentation object
trainAug = ImageDataGenerator(rotation_range=30, 
                              zoom_range=0.15, 
                              width_shift_range=0.2, 
                              height_shift_range=0.2, 
                              shear_range=0.15, 
                              horizontal_flip=True, 
                              fill_mode="nearest")

# Initialize the validation/testing data augmentation object 
valAug = ImageDataGenerator()

# Define the ImageNet mean subtraction (in RGB order) 
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean
n_epochs = 50

In [ ]:
X = tf.placeholder(shape = [None, 224, 224, 3], dtype = tf.float32, name = "X")

### The first layer will be composed of 256 maps of 104 x 104 capsules each.
Each capsule will output a 128 dimensional vector.

In [ ]:
caps_n_maps = 256
caps1_n_caps = caps1_n_maps * 104 * 104                                              # 2768896 capsules
caps1_n_dims = 128

In [ ]:
conv1 = tf.layers.Conv2D(X, name = "conv1", 
                        filters = 4096, 
                        kernel_size = 9,
                        strides = 1,
                        padding = "valid",
                        activation = tf.nn.relu)

conv2 = tf.layers.Conv2D(conv1, name = "conv2",
                        filters = caps1_n_maps * caps1_n_dims, 
                        kernel_size = 9, 
                        strides = 2,
                        padding = "valid",
                        activation = tf.nn.relu)

Since the kernel size is 9, the image is shrunk by (9 - 1 = 8) pixels after each Conv2D layer.

Hence, after two convolution layers we have (224, 224, 3) -> (216, 216, 3) -> (208, 208, 3).\
Moreover, as stride = 2, (208, 208, 3) -> (104, 104, 3)

### Output of the Conv2D layer:

Number of maps (256) * Vector dimensions per capsule (128) = 32768 feature maps for each capsule.\
Each feature map is 104 * 104.

In [ ]:
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims], name = "caps1_raw")

In [ ]:
def squash(s, axis = -1, epsilon = 1e-7, name = None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

In [ ]:
# Output of the first capsule layer
caps1_output = squash(caps1_raw, name="caps1_output")

### Double Digit Capsule Layer

In [ ]:
import os
import argparse

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image

from capsnet.twodigitcapsules.capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
from capsnet.twodigitcapsules.utils import combine_images, plot_log

keras.backend.set_image_data_format('channels_last')

In [ ]:
def CapsNet(input_shape, n_class, routings, primary_capsules=16, number_of_primary_channels=32, digit_capsules=16):
    """
    A Capsule Network on CIFAR.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x = keras.layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = keras.layers.Conv2D(filters=256, kernel_size=5, strides=2, padding='valid', name='conv1', kernel_regularizer=keras.regularizers.l2(1.e-4))(x)
    norm = keras.layers.BatchNormalization(axis=3)(conv1)
    conv1 = keras.layers.Activation('relu')(norm)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(conv1, dim_capsule=32, n_channels=16, kernel_size=5, strides=2, padding='valid')

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(primarycaps, dim_capsule=32, n_channels=16, kernel_size=5, strides=2, padding='valid', do_reshape=True)

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=digit_capsules, routings=routings, name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='capsnet')(digitcaps)

    # Decoder network.
    y = keras.layers.Input(shape=(n_class,))
    masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
    masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction

    # Shared Decoder model in training and prediction
    decoder = keras.models.Sequential(name='decoder')
    decoder.add(keras.layers.Dense(1024, activation='relu', input_dim=digit_capsules*n_class))
    decoder.add(keras.layers.Dense(1024, activation='relu'))
    decoder.add(keras.layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(keras.layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Models for training and evaluation (prediction)
    train_model = keras.models.Model([x, y], [out_caps, decoder(masked_by_y)])
    eval_model = keras.models.Model(x, [out_caps, decoder(masked)])

    # manipulate model
    noise = keras.layers.Input(shape=(n_class, digit_capsules))
    noised_digitcaps = keras.layers.Add()([digitcaps, noise])
    masked_noised_y = Mask()([noised_digitcaps, y])
    manipulate_model = keras.models.Model([x, y, noise], decoder(masked_noised_y))
    return train_model, eval_model, manipulate_model